In [8]:
import numpy as np
import random as rd
from sklearn.cluster import KMeans
import time


In [54]:
%run Utils.ipynb

The removers:

In [ ]:

    
def random_removal(solution, problem, q):
    """This function randomly removes q nodes from the solution. It returns the solution without the removed nodes, as well as the list of removed nodes.

    Args:
        solution (list): The solution containing nodes as a list of tuples.
        problem (dict): The problem dictionary containing nodes.
        q (int): The number of nodes to remove.
        
    Returns:
        tuple: A tuple containing the updated solution without removed nodes and the list of removed nodes.
    """
    
    nodes = problem["Nodes"]
    sol = solution.copy()
    
    to_remove = rd.sample(nodes, q)
    
    for node in to_remove:
        sol.remove(node)
    
    return sol, to_remove


In [ ]:
def largest_trip_removal(solution,problem,q):
    """This function picks q costly trips, and removes the node which is the "furthest" away. Will have to find a smart way to calculate the furthest node. 
    
    Initial thoughts is to find the 'center' of the trip, and pick the largest outlier except from the depot of cource. For instance when traveling from lagunen (d) to three nodes in bergen sentrum and one in
    bønes. The one in bønes is furthest away from the center even though it is close to the depot. Since there are multiple nodes in sentrum they pull heavy, and will pull the center towwards them

    PSEUDOCODE: 
    
    1. Loop q times
    2. Select the largest trip
    3. Look though the selected trip and find the node which increases the length the most. 
    4. Remove the node
    
      
    
    Args:
        solution (list): The generated solution
        problem (dict): Problem file
        q (int): amount of nodes to remove
        
    Returns:
        removed_sol (list): The solution without the removed nodes.
        removed_nodes (list): The removed nodes.
    """
    
    
    pp = problem["PreProseccing"]
    
    
    
    remove_list = []
    
    for _ in range(q):
        
        trips = split_solution(solution)
        
        trip_idx = 0
        length = 0
        
        
        
        # Find the largest trip
        for j in range(len(trips)-1):
            if len(trips[j]) > length:
                trip_idx =j
                length = len(trips[j])
        
        
        
        
        to_remove = None
        max_increase = 0
        
        trip = trips[trip_idx]
        
        padded = [(0,0)]+trip+[(0,0)]
        
        
        
        for i in range(1,len(padded)-1):
            p0 = padded[i-1]
            p1 = padded[i]
            p2 = padded[i+1]
            
            increase,_ = detour_length_pp(p0,p1,p2,pp)

            if increase > max_increase:
                to_remove = p1
                max_increase = increase
        
        if to_remove == None:
            continue
        else:
            
            remove_list.append(to_remove)
        
            solution.remove(to_remove)
    
    return solution, remove_list

In [2]:
def smallest_trip_removal(solution, problem, q):
    """_summary_

    Args:
        solution (_type_): _description_
        probelm (_type_): _description_
        q (_type_): _description_

    Returns:
        _type_: _description_
    """
    
    pp = problem["PreProseccing"]
    
    
    
    remove_list = []
    
    for _ in range(q):
        
        trips = split_solution(solution)
        
        trip_idx = len(trips)-1
        length = float("inf")
        
        
        
        # Find the largest trip
        for j in range(len(trips)-1):
            if len(trips[j])>0 and len(trips[j]) < length:
                trip_idx = j
                length = len(trips[j])
        
        
        
        
        to_remove = None
        max_increase = 0
        
        trip = trips[trip_idx]
        
        padded = [(0,0)]+trip+[(0,0)]
        
        
        
        for i in range(1,len(padded)-1):
            p0 = padded[i-1]
            p1 = padded[i]
            p2 = padded[i+1]
            
            increase,_ = detour_length_pp(p0,p1,p2,pp)

            if increase > max_increase:
                to_remove = p1
                max_increase = increase
        
        if to_remove == None:
            continue
        else:
            
            remove_list.append(to_remove)
        
            solution.remove(to_remove)
    
    return solution, remove_list

In [ ]:
def similarity_removal(solution, problem,q):
    """This function removes q nodes from the solution based on the similarity between the nodes. The nodes with the highest similarity are removed.
    It selects a node at random and removes the q-1 nodes that are most similar to it. 
    The similarity accounts for:
        -the location of the nodes
        -the number of crew
        -the distance from the depot
        -the distance from the nearest charging station
        

    Args:
        solution (_type_): _description_
        problem (_type_): _description_
        q (_type_): _description_
    """
    locations = problem["Locations"]
    nodes = problem["Nodes"]
    pp = problem["PreProseccing"]
    cs_matrix = pp["PreferredCS"]
    d_matrix = pp["Distances"] 
    dwc_matrix = pp["DistancesWithCharging"]
    jobs = problem["Jobs"]
    
    weights = {'location': 1.5, 'crew': 0.7, 'depot': 0.9, 'charging_station': 0.9, 'same task':0.6}
    
    
    chosen = rd.choice(nodes)
    loc_diff = []
    crew_diff = []
    depot_dist = []
    cs_dist = []
    same_task = []
    
    
    crew_0 = jobs[chosen[0]]
    depot_dist_0 = d_matrix[0][chosen[0]]
    cs_dist_0 = cs_matrix[chosen[0]][chosen[0]][1]
    
    for i in range(len(nodes)):
        node = nodes[i]
        crew_1 = jobs[node[0]]
        depot_dist_1 = d_matrix[0,node[0]]
        cs_dist_1 = cs_matrix[node[0]][node[0]][1]
        
        
        loc_diff.append(d_matrix[chosen[0],node[0]])
        crew_diff.append(abs(crew_0-crew_1))
        depot_dist.append(abs(depot_dist_0-depot_dist_1))
        cs_dist.append(abs(cs_dist_0-cs_dist_1))
        same_task.append(1 if node[1]==chosen[1] else 0)
    
    loc_diff = normalize(loc_diff)
    crew_diff = normalize(crew_diff)
    depot_dist = normalize(depot_dist)
    cs_dist = normalize(cs_dist)
    
    similarity = [sum(x) for x in zip(loc_diff,crew_diff,depot_dist,cs_dist)]
    
    similarity = [sum(x) for x in zip(np.multiply(loc_diff, weights['location']),
                                      np.multiply(crew_diff, weights['crew']),
                                      np.multiply(depot_dist, weights['depot']),
                                      np.multiply(cs_dist, weights['charging_station']),
                                      np.multiply(same_task,weights['same task']))]
    
    
    remove_list = []
    sol = solution.copy()
    
    for i in range(q):
        m = min(similarity) #indeksen til den mest like
        
        
        idx = similarity.index(m)
        similarity[idx] = 1000
        
        to_remove = nodes[idx]
       
        remove_list.append(to_remove)
        
        sol.remove(to_remove)
        
    
        
    return sol, remove_list

In [5]:

def clustering_removal_(solution,problem,k,q):
    """This function creates k clusters-centers using the k-means algorithm. It then removes the q nodes closest to a random cluster center

    Args:
        solution (_type_): _description_
        problem (_type_): _description_
        q (_type_): _description_
    """

    locations = problem["Locations"]
    n_cs = len(problem["ChargingStations"])

    coords = locations.values()
    ids = locations.keys()
    ids = list(ids)
    ids = ids[n_cs+1:]

    coords = list(coords)
    coords =  coords[n_cs+1:]
    coords = np.array(coords)

    
    kmeans = KMeans(n_clusters=k, tol = 0.001,max_iter=100).fit(coords)
    
    labels = kmeans.labels_
    
    clustercenters = kmeans.cluster_centers_
    
    order = rd.sample(range(k),k)
    
    toRemove = []
    
    while len(toRemove) < q:
        
        cluster = order.pop()
        
        for location, label in zip(ids, labels):
            if label == cluster:
                toRemove.append((location,0))
                toRemove.append((location,1))
            
    
    
    sol = solution.copy()
    
    for node in toRemove:
        sol.remove(node)
    
    return sol, toRemove



In [6]:
def clustering_removal(solution,problem,k,q):
    
    locations = problem["Locations"]
    
    to_remove = []
    
    while len(to_remove) < q:
            
            trips = split_solution(solution)
            
            idx = rd.randint(0,len(trips)-1)
            
            trip = trips[idx]
        
            if len(trip) == 0:
                continue
            
            ids = [node[0] for node in trip]
            
            ids = list(dict.fromkeys(ids))
            
            coords = np.array([locations[node] for node in ids])
            
    
            k = min(k,len(coords))
            kmeans = KMeans(n_clusters=k, tol = 0.001,max_iter=300).fit(coords)
            
            labels = kmeans.labels_
            
            clustercenters = kmeans.cluster_centers_
            
            order = rd.sample(range(k),k)
            
            for cluster in order:
                for id, label in zip(ids, labels):
                    if label == cluster:
                        if (id,0) in trip:
                            to_remove.append((id,0))
                            solution.remove((id,0))
                        if (id,1) in trip:
                            to_remove.append((id,1))
                            solution.remove((id,1))

    return solution, to_remove

In [1]:
def worst_removal(solution,problem,q):
    """This function removes the q most 'expensive' nodes from the solution. The most expensive nodes are the nodes that increase the length of the solution the most.
    To find these we loop through each trip, storing how much each node increases the length of the trip. We then remove the node that increases the length the most.
    In short we find how shorter A to C is compared to A to B to C, and remove B's where the margin is the largest.

    Args:
        solution (_type_): _description_
        problem (_type_): _description_
        q (_type_): _description_
    """
    
    
    pp = problem["PreProseccing"]
   
    
    trips = split_solution(solution)
    
    remove_list = []
    
    for trip in trips:
        
        padded = [(0,0)]+trip+[(0,0)]
        
        
        for i in range(1,len(padded)-1):
            p0 = padded[i-1]
            p1 = padded[i]
            p2 = padded[i+1]
            
            increase,_ = detour_length_pp(p0,p1,p2,pp)

            remove_list.append((p1,increase))
            
    remove_list = sorted(remove_list, key = lambda x: x[1], reverse = True)
    
    remove_list = [x[0] for x in remove_list][:q]
    
    for i in range(q):
        solution.remove(remove_list[i])
        
    
    return solution, remove_list

The inserters:

In [149]:
def greedy_insert(solution, to_insert, problem):
    """
    This function selects the greediest place to insert q removed nodes.
    The function checks all trips and inserts the node into the trip that will increase the distance the least.
    The node will not be placed in a trip if it breaks the passenger capacity and range of the largest vessel.

    Args:
        solution (list): The list of trips to insert the nodes into.
        to_insert (list): The nodes removed from the solution by the removal function.
        problem (dict): The problem file.

    Returns:
        list: The updated solution with the inserted nodes.
        
        
        
    ############
    Further work:
   
    """
    
    
    
    vesseltypes = problem["Vessels"]
    vessel = vesseltypes[max(vesseltypes.keys())]
    _, max_range, _, _, max_cap,_ = vessel
    pp = problem["PreProseccing"]
    
    sol,_ = allocate_vessels(solution,problem)
    vessels = sol[1]
    
    
    
    
    for node in to_insert:
        
        costs = insertion_cost(node,vessels,solution,problem)
        trip_starts = zeroIDX([(0,0)]+solution)
        if len(costs) == 0:
            solution.insert(trip_starts[-1]+1,node)
            continue
        
        insert = costs[0]
        
        _,t,p1 = insert

        
        if t == 0 and p1 == (0,0):
            idx = 0
        elif p1 == (0,0):
            idx = trip_starts[t]
            
        else:  
            idx = solution.index(p1)+1
        solution.insert(idx,node)
        
    return solution

In [1]:
def greedy_insert_old(solution, to_insert, problem):
    """
    This function selects the greediest place to insert q removed nodes.
    The function checks all trips and inserts the node into the trip that will increase the distance the least.
    The node will not be placed in a trip if it breaks the passenger capacity and range of the largest vessel.

    Args:
        solution (list): The list of trips to insert the nodes into.
        to_insert (list): The nodes removed from the solution by the removal function.
        problem (dict): The problem file.

    Returns:
        list: The updated solution with the inserted nodes.
        
        
        
    ############
    Further work:
   
    """
    
    
    
    vesseltypes = problem["Vessels"]
    vessel = vesseltypes[max(vesseltypes.keys())]
    _, max_range, _, _, max_cap,_ = vessel
    pp = problem["PreProseccing"]
    
    trips = split_solution(solution)
    
    for node in to_insert:
        lowest_increase = float("inf")
        modified_trip = 0
        chosen_trip = 0
        
      
        bounds = find_bounds(node,trips)
        
        zero_index = zeroIDX(solution)
        
        
        for t in range(len(trips)-1):
            
            trip = [(0,0)]+ trips[t] + [(0,0)]
            trip_start = zero_index[t]
            old_vessel, f = allocate_single_vessel(trips[t],problem)
            
            if old_vessel == 0:
                old_cost = 0
            else:
                
                old_cost = single_vessel_obj_func(trips[t],old_vessel,problem)
            
            
            lower, higher = 0, len(trip)-1
            
            
            for i in range(lower,higher):
                
                sol = solution.copy()
               
                p1 = trip[i]
                p2 = trip[i+1]
                
                new_trip,_ = detour(p1,node,trip)
                
                
                
                
                f,_ = feasible_trips((sol,0,0),problem)
                
                if f:
                
                    max_pass = single_max_pass(new_trip,problem)
                    
                    if max_pass <= max_cap:
                        
                        new_vessel, _ = allocate_single_vessel(new_trip[1:-1],problem) #add max pass
                        
                        new_cost = single_vessel_obj_func(new_trip[1:-1],new_vessel,problem)
                        
                        increase = new_cost - old_cost

                        if increase < lowest_increase:
                            lowest_increase,modified_trip,chosen_trip = increase,new_trip,t # -1 because of the depot
                        
            
        if modified_trip == 0:
            trips[-1].append(node)
        else:
            trips[chosen_trip] = modified_trip[1:-1]
        
        
    
    new_sol = combine_solution(trips)

    
    return new_sol

In [10]:

def insertion_cost(to_insert,vessels,solution,problem, k = None):
    """Function to help calculating the insertion costs

    Args:
        to_insert (_type_): _description_
        vessels (_type_): _description_
        solution (_type_): _description_
        problem (_type_): _description_
        k (_type_): _description_

    Returns:
        _type_: _description_
    """
    
    vesseltypes = problem["Vessels"]
    vessel = vesseltypes[max(vesseltypes.keys())]
    _, max_range, _, _, max_cap,_ = vessel
    
    node, binary = to_insert
    
    trips = split_solution(solution)
    
    
    
    costs = []
    
    for t in range(len(trips)-1):
        
        trip  = [(0,0)]+trips[t]+[(0,0)]
        
        old_vessel = vessels[t]
        
        if old_vessel == 0:
            old_cost = 0
        else: 
            old_cost = single_vessel_obj_func(trips[t],old_vessel,problem)
    
        lower, higher = 0, len(trip)-1
        
        
        
        for i in range(lower,higher):
            p1 = trip[i]
            
                        
            new_trip,_ = detour(p1,(node,binary),trip)
            
            
            
            
            f, new_vessel = feasible_insert(solution,to_insert,t,p1,new_trip,problem)
            
            if f:
                if new_vessel == 0:
                    print("ERROR")
            
                new_cost = single_vessel_obj_func(new_trip[1:-1],new_vessel,problem)
            
                increase = new_cost - old_cost
            
                costs.append((increase,t,p1))
                

    
    if k:
        sorted_list = sorted(costs,key = lambda x: x[0])[:min(k,len(costs)-1)]
    else: 
        sorted_list = sorted(costs,key = lambda x: x[0])
    
    
    return sorted_list

In [151]:
def kregret_insert(solution, to_insert,problem,k=5):
    """This is a k-regret greedy insertion. The function finds the cost of insertion for the removed nodes in all possible position. 
    Then calculating the k-regret for each node. regret = sorted_costs[k-1] - sorted_costs[0]. Then we loop through the nodes, starting with the largest regret and inserts it into the best postion.

    Args:
        solution (list): The list of trips to insert the nodes into.
        to_insert (list): The nodes removed from the solution by the removal function.
        problem (dict): The problem file.

    Returns:
        list: The updated solution with the inserted nodes.
    """    

    sol,_ = allocate_vessels(solution,problem)
    vessels = sol[1]
    
    regrets = []
    
    for node in to_insert:
        
        sorted_list = insertion_cost(node,vessels,solution,problem)
        
        
        if len(sorted_list) == 0:
            solution.append(node)
            continue
        
        regret = 0
        base_cost = sorted_list[0][0]
    
        for i in range(1,min(k,len(sorted_list))):
            regret += (sorted_list[i][0]-base_cost)
        
        
        regrets.append((regret,sorted_list,node))
        
    
    insertions = sorted(regrets,key = lambda x: x[0],reverse = True)
    
    
    for _,sorted_list,node in insertions:
        trip_starts = zeroIDX([(0,0)]+solution)
        
        trips = split_solution(solution)
        
        
        feasibility = False
        i = 0
        
        while not feasibility:
            
            if i >= len(sorted_list):
                solution.append(node)
                break
            
            _,t,p1 = sorted_list[i]
            
            trip = [(0,0)]+trips[t]+[(0,0)]
            
            new_trip, _ = detour(p1,node,trip)
            
            
            
            if t == 0 and p1 == (0,0):
                idx = 0
            elif p1 == (0,0):
                idx = trip_starts[t]
            else:  
                idx = solution.index(p1)+1
                
            f,_ = feasible_insert(solution,node,t,p1,new_trip,problem)
            
            if f:
                feasibility = True
                solution.insert(idx,node)
            i+=1
            
                    
        
    
    
    return solution

Remove-Insert pairs:

In [ ]:
def random_remove_greedy_insert(solution,problem,q):
    
    removed_sol, removed = random_removal(solution,problem,q)
    
    new_sol = greedy_insert(removed_sol,removed,problem)
    
    return new_sol

In [ ]:
def largest_trip_remove_greedy_insert(solution,problem,q):
    
    removed_sol, removed = largest_trip_removal(solution,problem,q)
    
    new_sol = greedy_insert(removed_sol,removed,problem)
    
    return new_sol

In [ ]:
def similarity_removal_greedy_insert(solution,problem,q):
    
    removed_sol, removed = similarity_removal(solution,problem,q)
    
    new_sol = greedy_insert(removed_sol,removed,problem)
    
    return new_sol

In [ ]:
def clustering_removal_kncs_greedy_insert(solution,problem,q):
    
    n_cs = len(problem["ChargingStations"])
    
    removed_sol, removed = clustering_removal(solution,problem,n_cs,q)
    
    new_sol = greedy_insert(removed_sol,removed,problem)
    
    return new_sol
    

In [7]:
def clustering_removal_krd_greedy_insert(solution,problem,q):
    
    n_locations = len(problem["Locations"])//15
    
    k = rd.choice(range(3,max(4,n_locations)))
    
    removed_sol, removed = clustering_removal(solution,problem,k,q)
    
    new_sol = greedy_insert(removed_sol,removed,problem)
    
    return new_sol

In [3]:
def random_remove_kregret_insert(solution,problem,q):
    
    removed_sol, removed = random_removal(solution,problem,q)
    
    new_sol = kregret_insert(removed_sol,removed,problem)
    
    return new_sol

In [4]:
def largest_trip_remove_kregret_insert(solution,problem,q):
    
    removed_sol, removed = largest_trip_removal(solution,problem,q)
    
    new_sol = kregret_insert(removed_sol,removed,problem)
    
    return new_sol


In [5]:
def similarity_removal_kregret_insert(solution,problem,q):
    
    removed_sol, removed = similarity_removal(solution,problem,q)
    
    new_sol = kregret_insert(removed_sol,removed,problem)
    
    return new_sol

In [14]:
def clustering_removal_kncs_kregret_insert(solution,problem,q):
    
    n_cs = len(problem["ChargingStations"])
    
    removed_sol, removed = clustering_removal(solution,problem,n_cs,q)
    
    new_sol = kregret_insert(removed_sol,removed,problem)
    
    return new_sol

In [13]:
def clustering_removal_krd_kregret_insert(solution,problem,q):
    
    n_locations = len(problem["Locations"])//15
    
    k = rd.choice(range(3,max(4,n_locations)))
    
    removed_sol, removed = clustering_removal(solution,problem,k,q)
    
    new_sol = kregret_insert(removed_sol,removed,problem)
    
    return new_sol

In [ ]:
def worst_removal_greedy_insert(solution,problem,q):
    
    removed_sol, removed = worst_removal(solution,problem,q)
    
    new_sol = greedy_insert(removed_sol,removed,problem)
    
    return new_sol

In [ ]:
def worst_removal_kregret_insert(solution,problem,q):
    
    removed_sol, removed = worst_removal(solution,problem,q)
    
    new_sol = kregret_insert(removed_sol,removed,problem)
    
    return new_sol

In [3]:
def smallest_trip_removal_greedy_insert(solution,problem,q):
    
    removed_sol, removed = smallest_trip_removal(solution,problem,q)
    
    new_sol = greedy_insert(removed_sol,removed,problem)
    
    return new_sol

In [4]:
def smallest_trip_removal_kregret_insert(solution,problem,q):
    
    removed_sol, removed = smallest_trip_removal(solution,problem,q)
    
    new_sol = kregret_insert(removed_sol,removed,problem)
    
    return new_sol

Escape operator:

In [148]:
def escape1_0(solution,problem,q):
    
    removed_sol, removed = random_removal(solution,problem,q)
    new_sol = send_to_dummy(removed_sol,removed)
        
        
    return new_sol

In [ ]:
def escape2_0(solution,problem,q):
    
    new_sol = random_remove_greedy_insert(solution,problem,q)
    
    return new_sol

Helpers:

In [ ]:
def split_solution(solution):
    sublists = []
    sublist = []
    
    for item in solution:
        if item == (0,0):
            sublists.append(sublist)
            sublist = []
        else:
            sublist.append(item)
    
    sublists.append(sublist)
    
    return sublists

def combine_solution(trips):
    combined_solution = []
    for trip in trips:
        combined_solution.extend(trip)
        combined_solution.append((0,0))
    
    if combined_solution and combined_solution[-1] == (0,0):
        combined_solution.pop()
    
    return combined_solution

In [ ]:
def detour_length_pp(p1,p2,p3,pp):
    
    distances = pp["Distances"]
    
    
    old_length = distances[p1,p2]
    new_length = distances[p1,p2] + distances[p2,p3]
    
    return new_length - old_length

def tourlength_pp(tour,pp):
    
    distances = pp["Distances"]
    tourlength = 0
    cum_length = [0]

    for i in range(len(tour)-1):
        p1 = tour[i][0]
        p2 = tour[i+1][0]
        tourlength += distances[p1,p2]
        cum_length.append(tourlength)
    
    return tourlength, cum_length
    
    

In [ ]:
def send_to_dummy(solution,calls):
    
    trips = split_solution(solution)
    
    trips[-1].extend(calls)
    
    solution = combine_solution(trips)
    
    return solution

In [ ]:
def normalize(array:list):
    max_val = max(array)
    min_val = min(array)
    
    norm = []
    
    for val in array:
        norm.append((val-min_val)/(max_val-min_val))
    
    return norm

In [15]:
def find_reverse_trip(trips,reverse):
    
    
    
    for trip in trips:
        if reverse in trip:
            return trip
        
    return None  

In [23]:
def find_bounds(to_insert, trips):
    
    bounds = []
    node,binary = to_insert
    
    if binary == 0:
        reverse = (node,1)
    else:
        reverse = (node,0)
        
    reverse_trip = find_reverse_trip(trips[:-1],reverse)
    
    for t in range(len(trips)-1):
        
        trip = [(0,0)]+trips[t]+[(0,0)]
        
        lower = 0
        higher = len(trip)-1
        
        if reverse_trip:
            print('here')
            if trips[t] == reverse_trip:
                print("Found a reverse trip")
                if binary == 0:
                    higher = trip.index(reverse)
                else:
                    lower = trip.index(reverse)+1
                    
                    
                bounds.append((lower,higher))
                continue
            
            conflicts = check_conflicting_nodes(trip,reverse_trip)
            
            if conflicts:
                for conflict in conflicts:
                    con, con_binary = find_node(trip,conflict)
                    
                    if binary == 1:
                    
                        if con_binary == 0:
                            lower = max(lower,trip.index((con, con_binary))+1)
                        
        bounds.append((lower,higher))

    return bounds

In [19]:
def find_node(trip,id):
    
    
    for node in trip:
        
        n,b = node
        if n == id:
            return node


In [150]:
def check_conflicting_nodes(trip1, trip2):
    # Extract just the node IDs from each trip
    nodes_trip1 = {node for node, _ in trip1}
    nodes_trip2 = {node for node, _ in trip2}
    
    # Find common nodes between the two trips
    common_nodes = nodes_trip1.intersection(nodes_trip2)
    
    conflicts = []
    
    # Check for each common node if the order of operations is feasible
    for node in common_nodes:
        # Check if both types of operations (0 and 1) exist for this node in either trip
        if any((node, 0) in trip1 and (node, 1) in trip2 for trip1, trip2 in [(trip1, trip2), (trip2, trip1)]):
            conflicts.append(node)
    
    return conflicts

In [15]:
def zeroIDX(lst):
    """
    Finds all indices where the element is (0, 0) in the given list.

    Args:
        lst (list): The list to search through.

    Returns:
        list: A list of indices where the element is (0, 0).
    """
    indices = [index for index, element in enumerate(lst) if element == (0, 0)]
    return indices

In [153]:

def feasible_insert(solution, node,trip,p1, new_trip,problem):
    
    vessel, feasible_allocation = allocate_single_vessel(new_trip[1:-1],problem)
    
    if feasible_allocation == False:
        return False, vessel
    
    
    
    #print(node)
    trips = split_solution(solution)
    
    trip = trips[trip]
    if p1== (0,0):
        idx = 0
    else:
        idx = trip.index(p1)+1
    
    reverse = (node[0],1-node[1])
    
    
    reverse_trip = find_reverse_trip(trips[:-1],reverse)
    
    if not reverse_trip:
        return True, vessel
    
    reverse_idx = reverse_trip.index(reverse)
    
    
    
    
    if reverse in trip:
        
        #print(f'Node: {node}')
        #print(f'Trip: {trip}')
        #print(f'Reverse: {reverse}')
        #print(f'Index: {idx}')
        if reverse[1] == 0:
            #print("Reverse == 0")
            if idx <= trip.index(reverse):
                return False, vessel
        
        else:
            if idx > trip.index(reverse)+1:
                return False, vessel
    else:
        
        
        if reverse[1] == 1:
            conflicts = check_conflicting_nodes(trip,reverse_trip)
            
            
            if len(conflicts)>1:
                
                for conflict in conflicts:
                    rev_con, rev_con_binary = find_node(reverse_trip,conflict)
                    rev_con_idx = reverse_trip.index((rev_con,rev_con_binary))
                

                    if rev_con_binary == 1:
                        if rev_con_idx < reverse_idx:
                            con, con_binary = rev_con, 0
                            con_idx = trip.index((con,con_binary))
                            if idx < con_idx:        
                                return False, vessel
                    
            if len(conflicts) == 1:
                
                
                rev_con, rev_con_binary = find_node(reverse_trip,conflicts[0])
                rev_con_idx = reverse_trip.index((rev_con,rev_con_binary))
                
                if rev_con_binary == 1:
                    if rev_con_idx < reverse_idx:
                          con, con_binary = rev_con, 0
                          con_idx = trip.index((con,con_binary))
                          if idx > con_idx:
                              return False, vessel
        elif reverse[1] == 0:
            conflicts = check_conflicting_nodes(trip,reverse_trip)
            
            
            if len(conflicts)>1:
                
                for conflict in conflicts:
                    rev_con, rev_con_binary = find_node(reverse_trip,conflict)
                    rev_con_idx = reverse_trip.index((rev_con,rev_con_binary))
                    
                    
                    if rev_con_binary == 1:
                        if rev_con_idx < reverse_idx:
                            con, con_binary = rev_con, 0
                            con_idx = trip.index((con,con_binary))
                            if idx > con_idx:
                                return False, vessel

                    
            elif len(conflicts) == 1:
                rev_con, rev_con_binary = find_node(reverse_trip,conflicts[0]) 
                rev_con_idx = reverse_trip.index((rev_con,rev_con_binary))  
                
                
                if rev_con_binary == 1: 
                    if rev_con_idx < reverse_idx:
                        con, con_binary = rev_con, 0
                        con_idx = trip.index((con,con_binary))
                        
                        if idx < con_idx:
                            return False, vessel
                        
    return True, vessel
    
    
    